In [1]:
import jax.numpy as jnp
import openferro as of
from openferro.units import Constants

We have imported openferro and jax.numpy. The latter works very similar to numpy. 

We also imported `Constants` from `openferro.units`. It contains some useful constants, e.g. the atomic mass unit in gram/mol, which can be got by `Constants.amu`.


# Initialization

We will define an on-lattice physical system in OpenFerro. 

A `of.system` object contains three pieces of information that specify a physical system.

- Lattice. 
- Local order parameters (we will use 'field' interchangeably to save ink) and global variables. 
- Hamiltonian. A Hamiltonian contains many interaction terms. We will add interaction terms to the physical system one by one.

These informations all have their own classes and a `of.system` object only store pointers to them. 

In the following, we will construct a simple cubic lattice, link a physical system to the lattice. Then we will define fields, global strain, and the Hamiltonian within the physical system. 

### Define lattice

In [2]:
## size of the lattice
L = 10
## Here we will just let the primitive vector be (4,0,0), (0,4,0) and (0,0,4)
latt_vecs = [jnp.array([4.0, 0.0, 0.0]), jnp.array([0.0, 4.0, 0.0]), jnp.array([0.0, 0.0, 4.0])]
## We define a 10X10X10 simple cubic lattice with given primitive vectors.
latt = of.SimpleCubic3D(L, L, L, latt_vecs[0], latt_vecs[1], latt_vecs[2])

We just defined a simple cubic lattice as a `SimpleCubic3D`(subclass of `BravaisLattice3D`).   
OpenFerro also supports other types of Bravais lattice. 

A few other options are 

- Body-centered cubic lattice (`of.BodyCenteredCubic3D`)

- Face-centered cubic lattice (`of.FaceCenteredCubic3D`)

- Hexagonal lattice (`of.Hexagonal3D`)

Then we link a `system` object to the lattice we just defined. At this point, the `system` object only contains the lattice information. It has an empty dictionary of fields, global variables, and interactions terms in the Hamiltonian. We need to add them one by one.

In [3]:
toy_system = of.System(latt)

### Define the fields and global variables

Each field linked to the system is an instance of `Field` class. They can be created by calling `add_field` method of the `system` object. 

The `add_field` method has the following arguments:

- ID: the name of the field. ID is unique so a system can not have two fields with the same ID. 
- ftype: the type of the field. Can be 'Rn' (general real-valued field with dimension n, e.g. local dipole moment), 'SO3' (orientation field, e.g. classical rigid spin), 'LocalStrain3D' (3D local strain field).
- dim: the dimension of the field, only used for 'Rn' field. 
- value: the initial value of the field. If left as None, the field will be initialized as default values. 
- mass: the mass of the field. Float scalar or array with same size as the lattice. For the latter, each site has a different mass for the field. 


In [4]:
## Add a dipole field  to the system. Each local dipole (u_i, i is site index) is a 3D vector with a mass of 100 amu. We set the initial value of all dipoles to be 0.
mass_of_dipole = 100 * Constants.amu
dipole_field = toy_system.add_field(ID="dipole", ftype="Rn", dim=3, value=0.0, mass = mass_of_dipole)
# ## Define the global strain
# mass_of_strain = Constants.amu * L**3  ## we make the mass of the global strain field scaling with the volume of the system. The prefactor is flexible. Here we simply set it to be atomic mass unit.
# ## The initial value of the global strain field is set to be [0.01, 0.01, 0.01, 0, 0, 0], correspond to a uniform strain of 1% in all three directions.
# gstrain  = toy_system.add_global_strain(value=jnp.array([0.01,0.01,0.01,0,0,0]), mass = mass_of_strain)

### Define the Hamiltonian

The Hamiltonian is defined by adding interaction terms to the system. 

There are two ways to add interaction terms to the system. 
- Add build-in interaction terms to the system. 
- Add custom interaction terms to the system. 

In this tutorial, we will first show how to add build-in interaction terms to the system. This can be easily done by calling methods of the `system` object. Then we will provide a minimal example of how to add custom interaction terms to the system, which is also easy if you are familiar with basic numpy operations.


#### Add build-in interaction terms to the system. 

**Onsite energy**

We first add the onsite self-interaction term to the system. 
The built-in onsite-interaction is $E=\sum_i K_2|u_i|^2+\alpha|u_i|^4+\gamma(u_{ix}^2u_{iy}^2 + u_{iy}^2u_{iz}^2 + u_{iz}^2u_{ix}^2)$. 

In [5]:
toy_system.add_dipole_onsite_interaction('self_onsite', field_ID="dipole", K2=-10.0, alpha=5.0, gamma=2)

**Dipole-Dipole interaction**

Dipole-dipole interaction is a long-range interaction.
With periodic boundary condition, the dipole-dipole interaction is given as 
$E_{dd} =\frac{1}{2} \frac{1}{4\pi\epsilon_0} \sum_{\textbf{n}} \sum_{i,j} \frac{u_i\cdot u_j }{|r_{ij}-a_n|^3} - \frac{3(u_i\cdot  (r_{ij}-a_n))(u_j\cdot (r_{ij}-a_n))}{|r_{ij}-a_n|^5}$. 
$n$ is the index of the periodic image. So $a_n$ is taken over all (infinitely many) the lattice vectors. 
$E_{dd}$ can be approximated by [Ewald summation](https://arxiv.org/abs/1811.09819). 

The dipole-dipole interaction can be added to the system by calling `add_dipole_dipole_interaction` method of the `system` object. 

The method has the following arguments:

- ID: the name of the interaction term. 
- field_ID: the ID of the field to which the interaction term is associated to. 
- prefactor: a scalar prefactor of the interaction term. So the interaction term added to the Hamiltonian is actually $\text{prefactor} \times E_{dd}$. The prefactor can be $1/\epsilon_r$ to deal with the dielectric constant. 


In [6]:
relative_dielectric_constant = 10.0
toy_system.add_dipole_dipole_interaction('dipole_ewald', field_ID="dipole", prefactor = 1 / relative_dielectric_constant )

#### Add Custom interaction terms to the system. 

A `system` object has three methods to add custum interaction terms to the Hamiltonian:

- `add_self_interaction`: add a custum interaction that only involves one field. This method requires the following arguments:
    - ID, field_ID, energy_engine, parameters
- `add_mutual_interaction`: add a custum interaction that involves two fields. This method requires the following arguments:
    - ID, field_1_ID, field_2_ID, energy_engine, parameters
- `add_triple_interaction`: add a custum interaction that involves three fields.
    - ID, field_1_ID, field_2_ID, field_3_ID, energy_engine, parameters

energy_engine is a plain Python function that takes the field values as arguments and returns the scalar energy of this interaction. parameters is a 1D array of parameters of the interaction. 

For example, `add_self_interaction` requires a energy_engine like
```python
def energy_engine(field_values: jnp.ndarray, parameters: jnp.ndarray):
    '''
    For example, for a R^d field defined on a 3D lattice, field_values is a 4D array with shape (L1, L2, L3, d). parameters is a 1D array.
    '''
    energy = some_scalar_function(field_values, parameters)
    return energy
```

`add_mutual_interaction` requires a energy_engine like
```python
def energy_engine(field_values_1: jnp.ndarray, field_values_2: jnp.ndarray, parameters: jnp.ndarray):
    energy = some_scalar_function(field_values_1, field_values_2, parameters)
    return energy
```

In the following, we add a custum nearest-neighbor interaction to the system, given as $E= \frac{J}{2} \sum_{i\sim j}  u_i \cdot u_j$. 

In [7]:
## First we define the energy engine.
def nn_interaction(field_values: jnp.ndarray, parameters):
    J = parameters[0]
    f = field_values
    energy = J * jnp.sum(f * jnp.roll(f, 1, axis=0))
    energy += J * jnp.sum(f * jnp.roll(f, 1, axis=1))
    energy += J * jnp.sum(f * jnp.roll(f, 1, axis=2))
    return energy
## Then we add the interaction to the system.
toy_system.add_self_interaction('nn_coupling', field_ID="dipole", energy_engine=nn_interaction, parameters=jnp.array([-1.0]))

And.. We are done! Isn't it easy? OpenFerro will handle all the others ( including GPU acceleration of evaluting custom interaction) for you. 

# Simulation

### Initialize the simulation

A `Simulation` object controls the work flow of the simulation. There are currently four types of subclasses of `Simulation` in OpenFerro (see [documentation](https://openferro.readthedocs.io/en/latest/theory-dynamics.html) for explaination):

- `MDMinimize`: Structural optimization.
- `SimulationNVE`: NVE dynamics.
- `SimulationNVTLangevin`: NVT dynamics with Langevin thermostat.
- `SimulationNPTLangevin`: NPT dynamics with Langevin thermostat.

We will use `SimulationNVTLangevin` in this tutorial. 


In [8]:
dt = 0.002 ## time step of the simulation is 0.002ps=2fs
temp = 300 ## temperature of the simulation is 300K
## Initialization of the simulation requires the system we just defined.
simulation = of.SimulationNVTLangevin(toy_system )
## Initialize the velocity of the local order parameters with a Gaussian distribution
simulation.init_velocity(mode='gaussian', temp=temp)
## Set the integrator of the field to be isothermal integrator, since we are doing NVT dynamics.
dipole_field.set_integrator('isothermal', dt=dt, temp=temp, tau=0.1)
## Add reporters to the simulation. The reporters will record the simulation data to files.
simulation.add_thermo_reporter(file='thermo.log', log_interval=100, potential_energy=True, kinetic_energy=True, temperature=True)  ## record the potential energy, kinetic energy, and temperature every 100 steps.
simulation.add_field_reporter(file_prefix='field', field_ID="dipole", log_interval=100,  field_average=True, dump_field=False) ## record the average value of the dipole field every 100 steps.

### Run the simulation for 5000 steps, amounting to 10ps. 

In [9]:
simulation.run(5000)

### Check the log files. Do you see the symmetry breaking? 